<a href="https://colab.research.google.com/github/SiyuanWuSFU/CM-T310-Artificial-Intelligence-Survey/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This file is used to download Kaggle datasets into a personal Google Drive.
In order to use this, we need a kaggle.json file, which contains the API credentials.
To obtain a kaggle.json file, sign into your Kaggle account and go to your profile, then 'My Account'. Under the API section, click 'Create New API Token' to generate the kaggle.json file. Afterwards, save it in the same directory as this .ipynb file.

In [ ]:
!pip install tensorflow_addons
!pip install tensorflow --upgrade


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import joblib as job
from sklearn.metrics import classification_report

In [3]:
from google.colab import drive
drive.mount('/content/drive')

dataset_df = pd.read_csv('/content/drive/MyDrive/9417project/train.csv')
labels = pd.read_csv('/content/drive/MyDrive/9417project/train_labels.csv')

Mounted at /content/drive


In [4]:
#Reduce Memory Usage
def get_minimal_dtype(df):

    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name
        if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
            if (col_type != 'object'):
                c_min = df[col].min()
                c_max = df[col].max()

                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)

                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        pass
            else:
                df[col] = df[col].astype('category')
    mem_usg = df.memory_usage().sum() / 1024**2
    print("Memory usage became: ",mem_usg," MB")

    return df
dataset_df = get_minimal_dtype(dataset_df)

Memory usage of dataframe is 4012.60 MB
Memory usage became:  1028.2593326568604  MB


In [ ]:
print("Full train dataset shape is {}".format(dataset_df.shape))


Full train dataset shape is (26296946, 20)


Data cleaning

In [5]:
missing_values = dataset_df.isnull().sum()

# Fill missing values
dataset_df['page'].fillna(-1, inplace=True)
dataset_df['room_coor_x'].fillna(-1, inplace=True)
dataset_df['room_coor_y'].fillna(-1, inplace=True)
dataset_df['screen_coor_x'].fillna(-1, inplace=True)
dataset_df['screen_coor_y'].fillna(-1, inplace=True)
dataset_df['hover_duration'].fillna(-1, inplace=True)

# Cap 'elapsed_time' at the 99th percentile
dataset_df['elapsed_time'] = dataset_df['elapsed_time'].clip(upper=dataset_df['elapsed_time'])

In [6]:
labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )
labels.head(5)



,session_id,correct,session,q
0,20090312431273200_q1,1,20090312431273200,1
1,20090312433251036_q1,0,20090312433251036,1
2,20090312455206810_q1,1,20090312455206810,1
3,20090313091715820_q1,0,20090313091715820,1
4,20090313571836404_q1,1,20090313571836404,1


In [ ]:
plt.figure(figsize=(20, 30))
plt.subplots_adjust(hspace=0.5, wspace=0.5)
for n in range(1,19):
    #print(n, str(n))
    ax = plt.subplot(6, 3, n)

    # filter df and plot ticker on the new subplot axis
    plot_df = labels.loc[labels.q == n]
    plot_df = plot_df.correct.value_counts()
    plot_df.plot(ax=ax, kind="bar", color=['b', 'c'])

    # chart formatting
    ax.set_title("Question " + str(n))
    ax.set_xlabel("")
plt.tight_layout()
plt.savefig("Distribution.png")

In [ ]:
print("Full label dataset shape is {}".format(labels.shape))

Full label dataset shape is (424116, 4)


Feature Engineering

In [ ]:
CATEGORICAL = ['event_name', 'name','fqid', 'room_fqid', 'text_fqid']
NUMERICAL = ['elapsed_time','level','page','room_coor_x', 'room_coor_y',
        'screen_coor_x', 'screen_coor_y', 'hover_duration']
def feature_engineer(dataset_df):
    dfs = []
    for c in CATEGORICAL:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('nunique')
        tmp.name = tmp.name + '_nunique'
        dfs.append(tmp)
    for c in NUMERICAL:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('mean')
        dfs.append(tmp)
    for c in NUMERICAL:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('std')
        tmp.name = tmp.name + '_std'
        dfs.append(tmp)
    dataset_df = pd.concat(dfs,axis=1)
    dataset_df = dataset_df.fillna(-1)
    dataset_df = dataset_df.reset_index()
    dataset_df = dataset_df.set_index('session_id')
    return dataset_df

In [ ]:
CATEGORICAL = ['event_name', 'name','fqid', 'room_fqid', 'text_fqid']
NUMERICAL = ['elapsed_time','level','page','room_coor_x', 'room_coor_y',
        'screen_coor_x', 'screen_coor_y', 'hover_duration']
BINNING = ['elapsed_time', 'room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y', 'hover_duration']

# Define feature engineering function
def feature_engineer_ver2(dataset_df):
    dfs = []
    for c in CATEGORICAL:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('nunique')
        tmp.name = c + '_nunique'
        dfs.append(tmp)

    for c in NUMERICAL:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('mean')
        tmp.name = c + '_mean'
        dfs.append(tmp)

        # Compute standard deviation only for certain features
        if c in BINNING:
            tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('std')
            tmp.name = c + '_std'
            dfs.append(tmp)

        # Binning
        if c in BINNING:  # Check if column is in the list of columns to bin
            dataset_df[c+'_bin'] = pd.qcut(dataset_df[c], q=4, duplicates='drop')
            tmp = dataset_df.groupby(['session_id','level_group'])[c+'_bin'].agg('count')
            tmp.name = c + '_bin_count'
            dfs.append(tmp)

    # Interaction between screen coordinates
    if 'screen_coor_x' in NUMERICAL and 'screen_coor_y' in NUMERICAL:
        # Compute Euclidean distance instead of product
        dataset_df['screen_coor'] = np.sqrt(dataset_df['screen_coor_x']**2 + dataset_df['screen_coor_y']**2)
        tmp = dataset_df.groupby(['session_id','level_group'])['screen_coor'].agg(['mean', 'std'])
        tmp.columns = ['screen_coor_mean', 'screen_coor_std']
        dfs.append(tmp)

    # Aggregated features
    if 'hover_duration' in NUMERICAL:
        dataset_df['total_hover_duration'] = dataset_df.groupby(['session_id'])['hover_duration'].transform('sum')
        tmp = dataset_df.groupby(['session_id','level_group'])['total_hover_duration'].agg('mean')
        tmp.name = 'total_hover_duration_mean'
        dfs.append(tmp)

    dataset_df = pd.concat(dfs,axis=1)
    dataset_df = dataset_df.fillna(-1)
    dataset_df = dataset_df.reset_index()
    dataset_df = dataset_df.set_index('session_id')
    return dataset_df


In [22]:
CATEGORICAL = ['event_name', 'name','fqid', 'room_fqid', 'text_fqid']
NUMERICAL = ['elapsed_time','level','page','room_coor_x', 'room_coor_y',
        'screen_coor_x', 'screen_coor_y', 'hover_duration']
BINNING = ['elapsed_time', 'room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y', 'hover_duration']

from sklearn.preprocessing import PowerTransformer

def feature_engineer_ver3(dataset_df):
    dfs = []
    pt = PowerTransformer(method='yeo-johnson')

    for c in CATEGORICAL:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('nunique')
        tmp.name = c + '_nunique'
        dfs.append(tmp)

        # Create dummy variables for top N most common events and names
        top_N = dataset_df[c].value_counts()[:10].index
        for val in top_N:
            dataset_df[c + '_' + val] = (dataset_df[c] == val).astype(int)
        tmp = dataset_df.groupby(['session_id','level_group']).agg({c + '_' + val: 'sum' for val in top_N})
        dfs.append(tmp)

    for c in NUMERICAL:
        # Fill missing values with the column median
        dataset_df[c].fillna(dataset_df[c].median(), inplace=True)

        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('mean')
        tmp.name = c + '_mean'
        dfs.append(tmp)

        # Compute standard deviation only for certain features
        if c in BINNING:
            tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('std')
            tmp.name = c + '_std'
            dfs.append(tmp)

        # Normalize 'elapsed_time' column
        if c == 'elapsed_time':
            dataset_df[c] = pt.fit_transform(dataset_df[[c]])

        # Binning
        if c in BINNING:  # Check if column is in the list of columns to bin
            dataset_df[c+'_bin'] = pd.qcut(dataset_df[c], q=4, duplicates='drop')
            tmp = dataset_df.groupby(['session_id','level_group'])[c+'_bin'].agg('count')
            tmp.name = c + '_bin_count'
            dfs.append(tmp)

    # Interaction between screen coordinates
    if 'screen_coor_x' in NUMERICAL and 'screen_coor_y' in NUMERICAL:
        # Compute Euclidean distance instead of product
        dataset_df['screen_coor'] = np.sqrt(dataset_df['screen_coor_x']**2 + dataset_df['screen_coor_y']**2)
        tmp = dataset_df.groupby(['session_id','level_group'])['screen_coor'].agg(['mean', 'std'])
        tmp.columns = ['screen_coor_mean', 'screen_coor_std']
        dfs.append(tmp)

    # Aggregated features
    if 'hover_duration' in NUMERICAL:
        dataset_df['total_hover_duration'] = dataset_df.groupby(['session_id'])['hover_duration'].transform('sum')
        tmp = dataset_df.groupby(['session_id','level_group'])['total_hover_duration'].agg('mean')
        tmp.name = 'total_hover_duration_mean'
        dfs.append(tmp)

    dataset_df = pd.concat(dfs, axis=1)
    dataset_df = dataset_df.fillna(-1)
    dataset_df = dataset_df.reset_index()
    dataset_df = dataset_df.set_index('session_id')

    return dataset_df

In [ ]:
CATEGORICAL = ['event_name', 'name', 'fqid', 'room_fqid', 'text_fqid']
NUMERICAL = ['elapsed_time', 'level', 'page', 'room_coor_x', 'room_coor_y',
             'screen_coor_x', 'screen_coor_y', 'hover_duration']

def feature_engineer_ver4(dataset_df):
    # Impute missing values
    # For numerical features use median
    for c in NUMERICAL:
        dataset_df[c].fillna(dataset_df[c].median(), inplace=True)

    # For categorical features use mode
    for c in CATEGORICAL:
        dataset_df[c].fillna(dataset_df[c].mode()[0], inplace=True)

    # Frequency encoding for 'event_name' and 'fqid'
    for c in ['event_name', 'fqid']:
        freq_encoding = dataset_df[c].value_counts(normalize=True)
        dataset_df[c + '_freq'] = dataset_df[c].map(freq_encoding)

    # Aggregate features for each session
    agg_features = dataset_df.groupby('session_id')[NUMERICAL].agg(['mean', 'sum', 'std'])
    agg_features.columns = ['_'.join(col).strip() for col in agg_features.columns.values]
    dataset_df = pd.merge(dataset_df, agg_features, on='session_id', how='left')

    # Interaction features
    dataset_df['screen_coor_interaction'] = dataset_df['screen_coor_x'] * dataset_df['screen_coor_y']
    dataset_df['level_page_interaction'] = dataset_df['level'] * dataset_df['page']

    # Frequency encoding for 'name', 'room_fqid', and 'text_fqid'
    for c in ['name', 'room_fqid', 'text_fqid']:
        freq_encoding = dataset_df[c].value_counts(normalize=True)
        dataset_df[c + '_freq'] = dataset_df[c].map(freq_encoding)
    dataset_df = dataset_df.drop(columns=['event_name'])
    dataset_df = dataset_df.drop(columns=['name'])
    dataset_df = dataset_df.reset_index()
    dataset_df = dataset_df.set_index('session_id')
    return dataset_df

In [ ]:
from sklearn.preprocessing import LabelEncoder

def feature_engineer_ver5(data):
    # Encode categorical variables
    categorical_cols = ['event_name', 'name', 'fqid', 'room_fqid', 'text_fqid']
    encoder = LabelEncoder()
    for col in categorical_cols:
        data[col] = encoder.fit_transform(data[col])

    # Generate historical features
    # Here we calculate the average elapsed_time for all previous events in the same session
    data['avg_elapsed_time'] = data.groupby('session_id')['elapsed_time'].transform('mean')

    # You can add more historical features as needed
    # For example, count the number of 'navigate_click' events for each session
    data['navigate_click_count'] = data[data['event_name'] == 'navigate_click'].groupby('session_id')['event_name'].transform('count')

    # Or calculate the maximum hover_duration for each session
    data['max_hover_duration'] = data.groupby('session_id')['hover_duration'].transform('max')
    # Count of each event type
    event_counts = data.groupby('session_id')['event_name'].value_counts().unstack(fill_value=0)
    data = data.join(event_counts, on='session_id')

    # Time since the last event
    data['time_since_last_event'] = data.groupby('session_id')['elapsed_time'].diff()

    # Number of events in fullscreen mode
    data['fullscreen_event_count'] = data[data['fullscreen'] == 1].groupby('session_id')['fullscreen'].transform('count')

    # Number of events with music on
    data['music_event_count'] = data[data['music'] == 1].groupby('session_id')['music'].transform('count')

    # Number of unique levels played
    data['unique_levels_count'] = data.groupby('session_id')['level'].transform('nunique')

    return data

In [24]:
# loading data by chunck to avoid mem running out
def partial_loader(df):
  split = 5
  epoch_length = df.shape[0]//split
  start = 0
  partial_df = df.iloc[start:start+epoch_length].copy(deep = True)
  partial_df = feature_engineer_ver3(partial_df)
  start += epoch_length
  for i in range(split - 2):
    partial_df = pd.concat([partial_df,feature_engineer_ver3(df.iloc[start:start+epoch_length].copy(deep = True))])
    start += epoch_length
  partial_df = pd.concat([partial_df,feature_engineer_ver3(df.iloc[start:])])
  return partial_df

In [ ]:
# select different version of feature engineer here

#dataset_df = feature_engineer(dataset_df)
#dataset_df = feature_engineer_ver2(dataset_df)
dataset_df = partial_loader(dataset_df)
#dataset_df = feature_engineer_ver4(dataset_df)
print("Full prepared dataset shape is {}".format(dataset_df.shape))

SVM model

In [26]:
# Create max_sub_model sub models, regular svm model if max_sub_model is equal to 1
def train_sub_models(train_df,train_users,train_labels,q_no):
  seed_value = 42
  max_countrol = False
  max_sub_model = 1

  print(train_labels.correct.value_counts())


  label_1 = train_labels.loc[train_labels.correct == 1 ]
  label_0 = train_labels.loc[train_labels.correct == 0 ]
  label_0_train_data = train_df[train_df.index.isin(label_0.index.values)]
  label_1_train_data = train_df[train_df.index.isin(label_1.index.values)]

  num_of_submodel = label_1.shape[0]//label_0.shape[0]
  if num_of_submodel > max_sub_model:
    num_of_submodel = max_sub_model
    max_countrol = True
  start = 0
  if num_of_submodel == 0:
    num_of_submodel = 1
  for i in range(num_of_submodel-1):
    model = SVC(probability=True)
    if max_countrol:
      split_label = label_1.iloc[start:start+label_1.shape[0]//(num_of_submodel)]
      concat_label = pd.concat([split_label,label_0])
      shuffled_label = concat_label.sample(frac=1, random_state=seed_value)
      shuffled_label = shuffled_label['correct']

      split_data = label_1_train_data.iloc[start:start+label_1_train_data.shape[0]//(num_of_submodel)]
      concat_data = pd.concat([split_data,label_0_train_data])
      shuffled_data = concat_data.sample(frac=1, random_state=seed_value)
      print(split_label.shape)
      model.fit(shuffled_data,shuffled_label)
      file_path = f'/content/drive/MyDrive/9417project/models/SVM_models/model_{grp}_{q_no}_sub{i}.joblib'
      job.dump(model,file_path)
      start = start + label_1.shape[0]//(num_of_submodel)
    else:
      split_label = label_1.iloc[start:start+label_0.shape[0]]
      concat_label = pd.concat([split_label,label_0])
      shuffled_label = concat_label.sample(frac=1, random_state=seed_value)
      shuffled_label = shuffled_label['correct']
      print(split_label.shape)

      split_data = label_1_train_data.iloc[start:start+label_0_train_data.shape[0]]
      concat_data = pd.concat([split_data,label_0_train_data])
      shuffled_data = concat_data.sample(frac=1, random_state=seed_value)

      model.fit(shuffled_data,shuffled_label)
      file_path = f'/content/drive/MyDrive/9417project/models/SVM_models/model_{grp}_{q_no}_sub{i}.joblib'
      job.dump(model,file_path)
      start = start + label_0.shape[0]


  split_label = label_1.iloc[start:]
  concat_label = pd.concat([split_label,label_0])
  shuffled_label = concat_label.sample(frac=1, random_state=seed_value)
  shuffled_label = shuffled_label['correct']

  split_data = label_1_train_data.iloc[start:]
  concat_data = pd.concat([split_data,label_0_train_data])
  shuffled_data = concat_data.sample(frac=1, random_state=seed_value)

  model = SVC(probability=True)

  model.fit(shuffled_data,shuffled_label)
  file_path = f'/content/drive/MyDrive/9417project/models/SVM_models/model_{grp}_{q_no}_sub{num_of_submodel-1}.joblib'

  job.dump(model,file_path)

  return num_of_submodel



In [40]:
# model training using cross validation
from sklearn.model_selection import KFold
# select number of folds for cv
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_indices, test_indices in k_fold.split(dataset_df.index.unique()):
  train_users, valid_users = dataset_df.index[train_indices], dataset_df.index[test_indices]
  train_x = dataset_df[dataset_df.index.isin(train_users)]
  valid_x = dataset_df[dataset_df.index.isin(valid_users)]
  model_per_question = []
  VALID_USER_LIST = valid_x.index.unique()


  prediction_df = pd.DataFrame(data=np.zeros((len(VALID_USER_LIST),18)), index=VALID_USER_LIST)


  for q_no in range(1,19):

      # Select level group for the question based on the q_no.
      if q_no<=3: grp = '0-4'
      elif q_no<=13: grp = '5-12'
      elif q_no<=22: grp = '13-22'
      print("### q_no", q_no, "grp", grp)


      # Filter the rows in the datasets based on the selected level group.
      train_df = train_x.loc[train_x.level_group == grp]
      train_users = train_df.index.values
      valid_df = valid_x.loc[valid_x.level_group == grp]
      valid_users = valid_df.index.values

      # Drop the 'correct' and 'level_group' columns from the features
      train_df = train_df.drop(columns=["level_group"])
      valid_df = valid_df.drop(columns=["level_group"])

      # Select the labels for the related q_no.

      train_labels = labels.loc[labels.q==q_no].set_index('session').loc[train_users]
      valid_labels = labels.loc[labels.q==q_no].set_index('session').loc[valid_users]

      valid_labels = valid_labels['correct']

      sub_model_number = train_sub_models(train_df.copy(),train_users.copy(),train_labels.copy(),q_no)
      model_per_question.append(sub_model_number)

  for q_no in range(1,19):
    if q_no<=3: grp = '0-4'
    elif q_no<=13: grp = '5-12'
    elif q_no<=22: grp = '13-22'
    for model_number in range(model_per_question[q_no-1]):
      model_path = f'/content/drive/MyDrive/9417project/models/SVM_models/model_{grp}_{q_no}_sub{model_number}.joblib'
      model = job.load(model_path)
      valid_pred = model.predict(valid_df)
      valid_pred_proba = model.predict_proba(valid_df)[:, 1]
      prediction_df.loc[valid_users, q_no-1] += (valid_pred_proba/model_per_question[q_no-1]).flatten()

  true_df = pd.DataFrame(data=np.zeros((len(VALID_USER_LIST),18)), index=VALID_USER_LIST)
  for i in range(18):
      # Get the true labels.
      tmp = labels.loc[labels.q == i+1].set_index('session').loc[VALID_USER_LIST]
      true_df[i] = tmp.correct.values

  max_score = 0; best_threshold = 0
  # loop through to find best threashold
  for threshold in np.arange(0.4,0.8,0.01):
      metric = tf.keras.metrics.F1Score(average="macro",threshold=threshold)
      y_true = tf.one_hot(true_df.values.reshape((-1)), depth=2)
      y_pred = tf.one_hot((prediction_df.values.reshape((-1))>threshold).astype('int'), depth=2)
      metric.update_state(y_true, y_pred)
      f1_score = metric.result().numpy()
      if f1_score > max_score:
          max_score = f1_score
          best_threshold = threshold
          best_pred = y_pred

  print("threshold ", best_threshold, "\tF1 score ", max_score)

### q_no 1 grp 0-4
1    5581
0    2211
Name: correct, dtype: int64
### q_no 2 grp 0-4
1    7640
0     152
Name: correct, dtype: int64
### q_no 3 grp 0-4
1    7266
0     526
Name: correct, dtype: int64
### q_no 4 grp 5-12
1    6164
0    1628
Name: correct, dtype: int64
### q_no 5 grp 5-12
1    4164
0    3628
Name: correct, dtype: int64
### q_no 6 grp 5-12
1    5954
0    1838
Name: correct, dtype: int64
### q_no 7 grp 5-12
1    5635
0    2157
Name: correct, dtype: int64
### q_no 8 grp 5-12
1    4693
0    3099
Name: correct, dtype: int64
### q_no 9 grp 5-12
1    5627
0    2165
Name: correct, dtype: int64
### q_no 10 grp 5-12
0    3905
1    3887
Name: correct, dtype: int64
### q_no 11 grp 5-12
1    4980
0    2812
Name: correct, dtype: int64
### q_no 12 grp 5-12
1    6685
0    1107
Name: correct, dtype: int64
### q_no 13 grp 5-12
0    5708
1    2084
Name: correct, dtype: int64
### q_no 14 grp 13-22
1    5374
0    2418
Name: correct, dtype: int64
### q_no 15 grp 13-22
0    4113
1    3679
Nam

In [28]:
# for q_no in range(1,19):
#   if q_no<=3: grp = '0-4'
#   elif q_no<=13: grp = '5-12'
#   elif q_no<=22: grp = '13-22'

#   # adding prediction probability from all sub_model
#   for model_number in range(model_per_question[q_no-1]):
#     model_path = f'/content/drive/MyDrive/9417project/models/SVM_models/model_{grp}_{q_no}_sub{model_number}.joblib'
#     model = job.load(model_path)
#     valid_pred = model.predict(valid_df)
#     valid_pred_proba = model.predict_proba(valid_df)[:, 1]
#     prediction_df.loc[valid_users, q_no-1] += (valid_pred_proba/model_per_question[q_no-1]).flatten()





In [43]:

# To save prediction file to local for easy access later #
file_path ='/content/drive/MyDrive/9417project/predictions/SVM_pred.npy'

np.save(file_path,prediction_df)
loaded_array = np.load(file_path,allow_pickle=True)
#prediction_df = np.load(file_path,allow_pickle=True)

In [44]:
## Optional running from this point ##
# get individual F1 for each question
prediction_df_label = prediction_df
print(prediction_df_label)

for i in range(18):
  prediction = prediction_df_label[i].values
  for j in range(len(prediction)):
    if prediction[j] >= best_threshold:
      prediction[j] = True
    else: prediction[j] = False
  zero_division  =   list(set(list(true_df[i].values)) - set(prediction))

  if len(zero_division)==1:
    report = classification_report(list(true_df[i].values),prediction,zero_division = int(zero_division[0]))
  else:
    report = classification_report(list(true_df[i].values),prediction)
  print(f'question {i+1}')
  print(report)

  #report_path = f'/content/drive/MyDrive/9417project/report/classification_report{i+1}.txt'
  #with open(report_path, 'w') as file:
      #file.write(report)

                    0    1    2    3    4    5    6    7    8    9    10   11  \
session_id                                                                      
20090313091715820  1.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  0.0  1.0  1.0   
20090313571836404  1.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  0.0  1.0  1.0   
20090314035813970  1.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  0.0  1.0  1.0   
20090314363702160  1.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  0.0  1.0  1.0   
20090315170769824  1.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  0.0  1.0  1.0   
...                ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
21020214413837144  1.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  0.0  1.0  1.0   
21020214491545340  1.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  0.0  1.0  1.0   
21020214510799336  1.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  0.0  1.0  1.0   
21020214532330784  1.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0  1.0  0.0  1.0  1.0   
21020214552027400  1.0  1.0 